In [1]:
#Andrew Weis
#PPHA 30531 Data Skills for Public Policy
#PSet4
#18 November 2019

#Import packages
import pandas as pd

In [2]:
PATH='/Users/andrewweis/Documents/GitHub/DataSkills/'
HW_FOLDER='assignment-4-awastro55'
DATA_FOLDER='chicago-police-data/data'
COMPLAINTS='/unified_data/complaints/'
DISCIPLINE='/context_data/'

In [3]:
#Generate the csv paths.
base_csv='complaints-{}_2000-2016_2016-11.csv'
datasets=['accused','victims','investigators']
discipline_file='discipline_penalty_codes.csv'

def csv_gen(sets,base):
    return base.format(sets)

complaints_files=[csv_gen(item, base_csv) for item in datasets]

In [4]:
def read_data(csv):
    complaints_data=[]
    for item in csv:
        complaints_data.append(pd.read_csv(PATH+DATA_FOLDER+COMPLAINTS+item))
        
    return complaints_data

In [5]:
complaints_data=read_data(complaints_files)
discipline=pd.read_csv(PATH+DATA_FOLDER+DISCIPLINE+discipline_file)
#Now we have our data frames.  

In [6]:
#Find data types
def check(df):
    return df.dtypes

check(discipline)

CODE             int64
ACTION_TAKEN    object
NOTES           object
dtype: object

In [7]:
check(complaints_data[0])

row_id                                       int64
complaints-accused_2000-2016_2016-11_ID    float64
cr_id                                        int64
complaint_category                          object
recommended_discipline                     float64
final_discipline                           float64
recommended_finding                         object
final_finding                               object
UID                                          int64
old_UID                                      int64
link_UID                                   float64
dtype: object

In [8]:
check(complaints_data[1])

cr_id       int64
gender     object
age       float64
race       object
dtype: object

In [9]:
check(complaints_data[2])

row_id                                             int64
complaints-investigators_2000-2016_2016-11_ID      int64
cr_id                                              int64
first_name                                        object
last_name                                         object
middle_initial                                    object
suffix_name                                       object
appointed_date                                    object
current_star                                     float64
current_rank                                      object
current_unit                                     float64
UID                                              float64
old_UID                                          float64
link_UID                                         float64
dtype: object

In [10]:
#Create a new column in a data frame.  We'll make copies of columns in the discipline frame.
def new_column(df, original, new):
    df[new]=df[original]
    return df

In [11]:
#Copy columns in discipline frame to get ready for merging.  
discipline=new_column(discipline,'CODE','recommended_discipline')
discipline=new_column(discipline,'CODE','final_discipline')
discipline=new_column(discipline,'ACTION_TAKEN','recommended_action')
discipline=new_column(discipline,'ACTION_TAKEN','final_action')
discipline

,CODE,ACTION_TAKEN,NOTES,recommended_discipline,final_discipline,recommended_action,final_action
0,0,VIOLATION NOTED,NaN,0,0,VIOLATION NOTED,VIOLATION NOTED
1,1,1 DAY SUSPENSION,NaN,1,1,1 DAY SUSPENSION,1 DAY SUSPENSION
2,2,2 DAY SUSPENSION,NaN,2,2,2 DAY SUSPENSION,2 DAY SUSPENSION
3,3,3 DAY SUSPENSION,NaN,3,3,3 DAY SUSPENSION,3 DAY SUSPENSION
4,4,4 DAY SUSPENSION,NaN,4,4,4 DAY SUSPENSION,4 DAY SUSPENSION
5,5,5 DAY SUSPENSION,NaN,5,5,5 DAY SUSPENSION,5 DAY SUSPENSION
6,6,6 DAY SUSPENSION,NaN,6,6,6 DAY SUSPENSION,6 DAY SUSPENSION
7,7,7 DAY SUSPENSION,NaN,7,7,7 DAY SUSPENSION,7 DAY SUSPENSION
8,8,8 DAY SUSPENSION,NaN,8,8,8 DAY SUSPENSION,8 DAY SUSPENSION
9,9,9 DAY SUSPENSION,NaN,9,9,9 DAY SUSPENSION,9 DAY SUSPENSION


In [20]:
#To make it clearer, pull out the data frames in complaints_data.
complaints_accused=complaints_data[0]
complaints_victims=complaints_data[1]
complaints_investigators=complaints_data[2]

In [27]:
#Now merge the data frames, in a few steps.
def merging():
    accrec=pd.merge(complaints_accused,discipline[['recommended_discipline','recommended_action']],
                    how='left',on='recommended_discipline')
    accnew=pd.merge(accrec,discipline[['final_discipline','final_action']], 
                how='left',on='final_discipline')
    accvic=pd.merge(accnew, complaints_data[1], how='outer', on='cr_id')
    accfinal=pd.merge(accvic,complaints_data[2][['cr_id', 'row_id',
                                             'complaints-investigators_2000-2016_2016-11_ID',
                                             'first_name','last_name',
                                             'middle_initial',
                                             'suffix_name',
                                             'appointed_date',
                                             'current_star',
                                             'current_rank',
                                             'current_unit']],how='outer',on='cr_id',
                     suffixes=('_acc','_inv'))
    return accfinal

In [28]:
accfinal=merging()
accfinal

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [30]:
final_complaint=accfinal.drop_duplicates()
final_complaint

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [31]:
final_complaint.to_csv(PATH+'complaint.csv')

In [35]:
#Group by
by_crid=final_complaint.groupby(['complaint_category', 'final_finding']).nunique('cr_id')

In [36]:
by_crid
#I have to submit so just looking over the list--complaints about intoxication, 
#especially on duty; DUI; positive drug test are the likeliest to be sustained.  

row_id_acc  \
complaint_category                                 final_finding               
01A-USE OF PROFANITY                               EX                     44   
                                                   NAF                    84   
                                                   NS                   2152   
                                                   SU                     62   
                                                   UN                    707   
01B-RACIAL/ETHNIC, ETC.                            EX                      3   
                                                   NAF                   263   
                                                   NS                    655   
                                                   SU                     36   
                                                   UN                    168   
01C-MISCELLANEOUS                                  EX                     25   
                                                   NAF                   111   
                                                   NS                    376   
                                                   SU                     12   
                                                   UN                    156   
02A-INTOXICATED ON DUTY                            NAF                     3   
                                                   NS                      5   
                                                   SU                     28   
                                                   UN                     22   
02B-INTOXICATED OFF DUTY                           EX                      1   
                                                   NAF                     3   
                                                   NS                     14   
                                                   SU                     54   
                                                   UN                     10   
02C-D.U.I. - ON DUTY                               NAF                     2   
                                                   UN                      2   
02D-D.U.I. - OFF DUTY                              NS                      2   
                                                   SU                    113   
                                                   UN                      3   
02E-POSSESSION/DRINKING ALCOHOL - ON DUTY          NAF                     4   
...                                                                      ...   
14B-FEDERAL CIVIL SUIT                             NS                    133   
                                                   SU                      4   
                                                   UN                    319   
15A-USE/ABUSE DRUGS/CONTR. SUBSTANCE - ON DUTY     NS                      1   
                                                   SU                      6   
                                                   UN                     15   
15B-USE/ABUSE DRUGS/CONTR. SUBSTANCE - OFF DUTY    NAF                     3   
                                                   NS                     10   
                                                   SU                      4   
                                                   UN                     35   
15C-D.U.I., DRUGS/ CONTR. SUB. - ON DUTY           NS                      1   
                                                   SU                      1   
                                                   UN                      2   
15D-D.U.I., DRUGS/ CONTR. SUB. - OFF DUTY          EX                      1   
                                                   NAF                     1   
                                                   SU                      3   
                                                   UN                      1   
15E-POSITIVE DRUG SCREEN - ORIGINATED FROM COMP... EX             

In [38]:
by_crid('final_finding').value_counts()

TypeError: 'DataFrame' object is not callable

In [42]:
print(final_complaint['final_finding'].value_counts())
fraction_sustained=9905/(9905+47066+45996+34918+9742)
print(fraction_sustained)
#Only 6.7% of complaints get sustained.  

NS     47066
NAF    45996
UN     34918
SU      9905
EX      9742
NC        60
DIS       12
Name: final_finding, dtype: int64
0.06709477263644184


In [43]:
by_race=final_complaint.groupby(['race', 'final_finding']).nunique('cr_id')

In [44]:
by_race
#It looks like, indepe

row_id_acc  \
race                           final_finding               
ASIAN/PACIFIC ISLANDER         EX                      8   
                               NAF                    84   
                               NS                     45   
                               SU                     17   
                               UN                     24   
BLACK                          EX                    581   
                               NAF                  8349   
                               NS                   3274   
                               SU                    332   
                               UN                   1832   
HISPANIC                       EX                     76   
                               NAF                  1468   
                               NS                    702   
                               SU                    120   
                               UN                    360   
NATIVE AMERICAN/ALASKAN NATIVE EX                      1   
                               NAF                     2   
                               NS                      3   
                               SU                      3   
                               UN                      2   
WHITE                          EX                     86   
                               NAF                  1264   
                               NS                    405   
                               SU                    140   
                               UN                    263   

                                              complaints-accused_2000-2016_2016-11_ID  \
race                           final_finding                                            
ASIAN/PACIFIC ISLANDER         EX                                                   8   
                               NAF                                                 84   
                               NS                                                  45   
                               SU                                                  17   
                               UN                                                  24   
BLACK                          EX                                                 547   
                               NAF                                               4701   
                               NS                                                2268   
                               SU                                                 321   
                               UN                                                1559   
HISPANIC                       EX                                                  73   
                               NAF                                               1234   
                               NS                                                 627   
                               SU                                                 118   
                               UN                                                 343   
NATIVE AMERICAN/ALASKAN NATIVE EX                                                   1   
                               NAF                                                  2   
                               NS                                                   3   
                               SU                                                   3   
                               UN                                                   2   
WHITE                          EX                                                  86   
                               NAF                                               1128   
                               NS                                                 386   
                               SU                                                 136   
                               UN                                                 260   

                     